In [1]:
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.metrics import Mean,SparseCategoricalAccuracy

In [2]:
def get_mnist_ds():
    (train_validation_ds, test_ds) ,ds_info = tfds.load(name='mnist',
                                                        shuffle_files=True,
                                                        as_supervised=True,
                                                        split=['train','test'],
                                                               with_info=True)
    n_train_validation = ds_info.splits['train'].num_examples

    train_ratio = 0.8
    n_train = int(n_train_validation * train_ratio)
    n_validation = n_train_validation -n_train

    train_ds = train_validation_ds.take(n_train)
    remaning_ds = train_validation_ds.skip(n_train)
    validation_ds = remaning_ds.take(n_validation)

    return train_ds,validation_ds,test_ds

train_ds,validation_ds,test_ds=get_mnist_ds()

In [3]:
validation_ds

<TakeDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [4]:
def standardization(TRAIN_BATCH_SIZE,TEST_BATCH_SIZE):
    global train_ds, validation_ds, test_ds

    def stnd(images, labels):
        images = tf.cast(images, tf.float32) /255.
        return [images, labels]
    
    train_ds = train_ds.map(stnd).shuffle(1000).batch(TRAIN_BATCH_SIZE)
    validation_ds = validation_ds.map(stnd).batch(TEST_BATCH_SIZE)
    test_ds = test_ds.map(stnd).batch(TEST_BATCH_SIZE)



In [6]:
def load_metrics():
    global train_loss, train_acc
    global validation_loss, validation_acc
    global test_loss, test_acc

    train_loss = Mean()
    validation_loss = Mean()
    test_loss = Mean()

    train_acc = SparseCategoricalAccuracy()
    validation_acc = SparseCategoricalAccuracy()
    test_acc = SparseCategoricalAccuracy()

In [7]:
@tf.function
def trainer():
    global train_ds, model, loss_object, optimizer
    global train_loss, train_acc
    for images, labels in train_ds:
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_acc(labels, predictions)

In [8]:
@tf.function
def validation():
    global validation_ds, model, loss_object
    global validation_loss, validation_acc
    for images, labels in validation_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        validation_loss(loss)
        validation_acc(labels, predictions)

In [9]:
@tf.function
def tester():
    global test_ds, model, loss_object
    global test_loss, test_acc
    for images, labels in test_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        test_loss(loss)
        test_acc(labels, predictions)
'''
    template ='test Loss: {:.4f}\t test accuracy:{:.2f}%\n'

    print(template.format(test_loss.result(),
                          test_acc.result() * 100))
'''

"\n    template ='test Loss: {:.4f}\t test accuracy:{:.2f}%\n'\n\n    print(template.format(test_loss.result(),\n                          test_acc.result() * 100))\n"

In [10]:
def train_reporter():
    global epoch
    global train_loss, train_acc
    global validation_loss, validation_acc
    #on 
    print(colored('Epoch','red','on_white'), epoch+1)

    template = 'Train Loss: {:.4f}\t Train Accuracy:{:.2f}%\n' +\
    'validation Loss: {:.4f}\t valiation accuracy:{:.2f}%\n'

    print(template.format(train_loss.result(),
                          train_acc.result() * 100,
                          validation_loss.result(),
                          validation_acc.result() * 100))
    train_acc.reset_states()
    train_loss.reset_states()
    validation_loss.reset_states()
    validation_acc.reset_states()

In [11]:
EPOCHS = 10
LR = 0.001
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 32

standardization(TRAIN_BATCH_SIZE,TEST_BATCH_SIZE)


In [6]:
'''
model = MNIST_Classfier()
loss_object = SparseCategoricalCrossentropy()
optimizer = SGD(learning_rate=LR)

load_metrics()

for epoch in range(EPOCHS):
    trainer()
    validation()
    train_reporter()

tester()
'''

'\nmodel = MNIST_Classfier()\nloss_object = SparseCategoricalCrossentropy()\noptimizer = SGD(learning_rate=LR)\n\nload_metrics()\n\nfor epoch in range(EPOCHS):\n    trainer()\n    validation()\n    train_reporter()\n\ntester()\n'

In [10]:
conv = Conv2D(filters=8, kernel_size=3, padding='valid',activation='relu')
pool = MaxPooling2D(pool_size=2, strides=2)

image = tf.random.normal(mean=0, stddev = 1, shape=(1,28,28,1))
print(image.shape)
conved = conv(image)
print(conved.shape)

print('w: ',conv.get_weights()[0].shape)
print('b: ',conv.get_weights()[1].shape)

(1, 28, 28, 1)
(1, 26, 26, 8)
w:  (3, 3, 1, 8)
b:  (8,)


In [9]:
pooled = pool(conved)
print(pooled.shape)

(1, 13, 13, 8)
